# AutoEncoder

#### Recommender System (Rating out of 5)

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

## Part 1 - Data Preprocessing

### Data Loading

In [2]:
BASE_PATH = "../../../Deep_Learning_Extended/Boltzmann_Machines/"
MOVIES_PATH = BASE_PATH + "ml-1m/movies.dat"
USERS_PATH = BASE_PATH + "ml-1m/users.dat"
RATINGS_PATH = BASE_PATH + "ml-1m/ratings.dat"

In [3]:
movies = pd.read_csv(MOVIES_PATH, sep="::", header=None, engine='python', 
                    encoding="latin-1")

In [4]:
users = pd.read_csv(USERS_PATH, sep="::", header=None, engine='python', 
                    encoding="latin-1")

In [5]:
ratings = pd.read_csv(RATINGS_PATH, sep="::", header=None, engine='python', 
                    encoding="latin-1")

### Preparing the Training Set and Test Set

In [6]:
TRAIN_PATH = BASE_PATH + "ml-100k/u1.base"
TEST_PATH = BASE_PATH + "ml-100k/u1.test"

In [7]:
training_set_df = pd.read_csv(TRAIN_PATH, delimiter='\t')
training_set = np.array(training_set_df, dtype='int')

In [8]:
test_set_df = pd.read_csv(TEST_PATH, delimiter='\t')
test_set = np.array(training_set_df, dtype='int')

### Matrix

In [9]:
nb_users  = int(max(max(training_set[:, 0]),max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1]),max(test_set[:, 1])))

In [10]:
def convert_to_matrix(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies  = data[:, 1][data[:, 0] == id_users]
        id_ratings = data[:, 2][data[:, 0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data

In [11]:
training_set = convert_to_matrix(training_set)
test_set = convert_to_matrix(test_set)

### Converting into Torch Tensor

In [12]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

## Part 2 - AutoEncoder

### Stacked AutoEncoder Architecture

In [13]:
class SAE(nn.Module):
    def __init__(self):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
        
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x

### SAE Model

In [14]:
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr=0.01, weight_decay=0.5)

### Training the SAE

In [15]:
epochs = 200
for epoch in range(1, epochs+1):
    train_loss = 0
    counter = 0.
    for id_user in range(0, nb_users):
        input_data = Variable(training_set[id_user]).unsqueeze(0)
        target = input_data.clone()
        if (torch.sum(target.data > 0) > 0):
            output = sae(input_data)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data * mean_corrector)
            counter += 1.
            optimizer.step()
    print("Epoch: {}/{} \t Loss: {}".format(epoch, epochs, train_loss/counter))

Epoch: 1/200 	 Loss: 1.77183198928833
Epoch: 2/200 	 Loss: 1.0967309474945068
Epoch: 3/200 	 Loss: 1.0532041788101196
Epoch: 4/200 	 Loss: 1.0383158922195435
Epoch: 5/200 	 Loss: 1.0309909582138062
Epoch: 6/200 	 Loss: 1.0264800786972046
Epoch: 7/200 	 Loss: 1.0239206552505493
Epoch: 8/200 	 Loss: 1.0218594074249268
Epoch: 9/200 	 Loss: 1.020734190940857
Epoch: 10/200 	 Loss: 1.0196418762207031
Epoch: 11/200 	 Loss: 1.0188077688217163
Epoch: 12/200 	 Loss: 1.018248438835144
Epoch: 13/200 	 Loss: 1.017877459526062
Epoch: 14/200 	 Loss: 1.0175063610076904
Epoch: 15/200 	 Loss: 1.0172780752182007
Epoch: 16/200 	 Loss: 1.0169247388839722
Epoch: 17/200 	 Loss: 1.016385793685913
Epoch: 18/200 	 Loss: 1.0165770053863525
Epoch: 19/200 	 Loss: 1.0164834260940552
Epoch: 20/200 	 Loss: 1.0161021947860718
Epoch: 21/200 	 Loss: 1.0159554481506348
Epoch: 22/200 	 Loss: 1.0158191919326782
Epoch: 23/200 	 Loss: 1.0158555507659912
Epoch: 24/200 	 Loss: 1.0158730745315552
Epoch: 25/200 	 Loss: 1.0157117

## Part 3 - Prediction

In [18]:
test_loss = 0
counter = 0.
for id_user in range(0, nb_users):
    input_data = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user]).unsqueeze(0)
    if (torch.sum(target.data > 0) > 0):
        output = sae(input_data)
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data * mean_corrector)
        counter += 1.
        optimizer.step()
print("Test Loss: {}".format(test_loss/counter))

Test Loss: 0.9236008524894714
